In [1]:
import pandas as pd
from tqdm import tqdm

In [135]:
#uploading players' stats
data = pd.read_csv("../data/player_stats.csv")
data.drop("Unnamed: 0",axis=1,inplace=True)

In [136]:
#uploadings ballon d'or rankings
ballonsdor = {}
for year in range(1996,2020):
    ballonsdor[year] = pd.read_csv("../data/bo_"+str(year)+".csv")

In [137]:
#forcing int type for points column
for year in tqdm(range(1996,2020)):
   try:
    ballonsdor[year]["Points"] = ballonsdor[year]["Points"].astype(int)
   except KeyError:
     print("Points column not found in "+str(year)+", proceeding")

100%|██████████| 24/24 [00:00<00:00, 1860.52it/s]

Points column not found in 2010, proceeding
Points column not found in 2011, proceeding
Points column not found in 2012, proceeding
Points column not found in 2013, proceeding
Points column not found in 2014, proceeding


In [138]:
#normalizing points won for each year
for df in tqdm(ballonsdor.keys()):
  try:
    ballonsdor[df]["%"] = ballonsdor[df]["Points"]*100/ballonsdor[df]["Points"].sum()
  except KeyError:
    print("Points column not found in "+str(df)+", proceeding")
  except TypeError:
    continue

100%|██████████| 24/24 [00:00<00:00, 2125.99it/s]

Points column not found in 2010, proceeding
Points column not found in 2011, proceeding
Points column not found in 2012, proceeding
Points column not found in 2013, proceeding
Points column not found in 2014, proceeding


In [139]:
#creating join key for player's stats
data["BO_JK"] = data["Season_End_Year"].astype(str) + data["Player"]

In [140]:
#creating join keys for BO rankings
for year in tqdm(ballonsdor):
    try:
        ballonsdor[year]["BO_JK"] = str(year) + ballonsdor[year]["Nom"]
    except KeyError:
        ballonsdor[year]["BO_JK"] = str(year) + ballonsdor[year]["Joueur"]

100%|██████████| 24/24 [00:00<00:00, 1717.89it/s]


In [141]:
#normalizing column names
for year in ballonsdor:
    try:    
        ballonsdor[year]["Nom"] = ballonsdor[year]["Joueur"]
        ballonsdor[year].drop("Joueur",axis=1,inplace=True)
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["%"] = ballonsdor[year]["Pourcentage"]
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Équipe nationale"]
    except KeyError:
        continue

for year in ballonsdor:
    try:
        ballonsdor[year]["%"] = ballonsdor[year]['Pourcentage[6]']
    except KeyError:
        continue
for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Sélection nationale"]
    except KeyError:
        continue
for year in ballonsdor:
    try:
        ballonsdor[year]["Pays"] = ballonsdor[year]["Nationalité"]
    except KeyError:
        continue


In [142]:
#append BO rankings together
BO_rankings = ballonsdor[1996].head()
BO_rankings.drop(0,inplace=True)
for year in ballonsdor:
    BO_rankings = BO_rankings.append(ballonsdor[year])

C:\Users\charl\miniconda3\envs\BallondOr\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [143]:
BO_rankings.isna().sum()

Rang                    46
Nom                      0
Pays                     0
Club                     0
Points                 155
Pos 1                  683
Pos 2                  676
Pos 3                  665
Pos 4                  663
Pos 5                  652
%                       13
BO_JK                    0
Sélection nationale    263
Nationalité            541
Rang[6]                686
Pourcentage[6]         686
Pourcentage            590
Unnamed: 0             686
Équipe nationale       686
Poste                  649
dtype: int64

In [147]:
#drop useless/duplicate columns
BO_rankings = BO_rankings[["Rang","BO_JK","Nom","Pays","Club","%"]]

In [145]:
BO_rankings

,Rang,BO_JK,Nom,Pays,Club,%
1,2.0,1996Ronaldo,Ronaldo,Brésil,PSV Eindhoven / FC Barcelone,18.69281
2,3.0,1996Alan Shearer,Alan Shearer,Angleterre,Blackburn Rovers / Newcastle United,13.986928
3,4.0,1996Alessandro Del Piero,Alessandro Del Piero,Italie,Juventus,9.019608
4,5.0,1996Jürgen Klinsmann,Jürgen Klinsmann,Allemagne,Bayern Munich,7.843137
0,1.0,1996Matthias Sammer,Matthias Sammer,Allemagne,Borussia Dortmund,18.823529
...,...,...,...,...,...,...
25,26.0,2019Karim Benzema,Karim Benzema,France,Real Madrid,0.035511
26,26.0,2019Georginio Wijnaldum,Georginio Wijnaldum,Pays-Bas,Liverpool FC,0.035511
27,28.0,2019João Félix,João Félix,Portugal,Benfica Lisbonne Atlético de Madrid,0.0
28,28.0,2019Marquinhos,Marquinhos,Brésil,Paris Saint-Germain,0.0


In [148]:
#joins
for year in tqdm(ballonsdor):
    new_data = data.merge(right=BO_rankings,on="BO_JK",how="right")

100%|██████████| 24/24 [00:00<00:00, 36.55it/s]


In [149]:
new_data.dropna(subset=["Season_End_Year"],inplace=True)
new_data["Season_End_Year"] = new_data["Season_End_Year"].astype(int)

1) créer clés squad_JK et country_JK dans new_data
2) pour chaque .csv de cup winners, créer clés cup_JK_1 (winner) et cup_JK_2 (finalist)
3) pour chaque cup de club, créer une colonne qui vaut 1 si la clé est trouvée, 0 sinon
4) idem pour cup nationales

In [150]:
#1) créer clés squad_JK et Nation_JK dans new_data
new_data["Squad_JK"] = new_data["Season_End_Year"].astype(str) + new_data["Squad"]
new_data["Nation_JK"] = new_data["Season_End_Year"].astype(str) + new_data["Nation"]

In [126]:
competitions_winners = {}

In [151]:
competitions_winners["Bundesliga"] = pd.read_csv("../data/Bundesliga_winners.csv")
competitions_winners["C3"] = pd.read_csv("../data/C3_Winners.csv")
competitions_winners["UCL"] = pd.read_csv("../data/Champions_League_winners.csv")
competitions_winners["Club WC"] = pd.read_csv("../data/club_world_cup_winners.csv")
competitions_winners["Copa America"] = pd.read_csv("../data/copa_america_winners.csv")
competitions_winners["Euro"] = pd.read_csv("../data/euro_winners.csv")
competitions_winners["Liga"] = pd.read_csv("../data/Liga_winners.csv")
competitions_winners["Ligue 1"] = pd.read_csv("../data/ligue1_winners.csv")
competitions_winners['PL'] = pd.read_csv("../data/PL_winners.csv")
competitions_winners['Serie A'] = pd.read_csv("../data/SerieA_winners.csv")
competitions_winners['WC'] = pd.read_csv('../data/WC_winners.csv')

In [152]:
competitions_winners["Ligue 1"]["Winner"] = competitions_winners["Ligue 1"][" Winner"]
competitions_winners["Ligue 1"].drop(" Winner",axis=1,inplace=True)

In [153]:
#2) pour chaque .csv de cup winners, créer clés cup_JK_1 (winner) et cup_JK_2 (finalist)
for competition in tqdm(competitions_winners.items()):
    competitions_winners[competition[0]][competition[0]+"_JK_1"] = (competitions_winners[competition[0]]["Year"].astype(str) + competitions_winners[competition[0]]["Winner"]).apply(lambda x : x.strip())
    try:
        competitions_winners[competition[0]][competition[0]+"_JK_2"] = (competitions_winners[competition[0]]["Year"].astype(str) + competitions_winners[competition[0]]["Finalist"]).apply(lambda x : x.strip())
    except KeyError:
        continue
    

100%|██████████| 11/11 [00:00<00:00, 410.92it/s]


In [154]:
#3) pour chaque cup de club, créer une colonne dans new_data qui vaut 1 si la clé est trouvée, 0 sinon
#doesn't work
for competition in tqdm(competitions_winners.items()):
    new_data["winner_"+competition[0]] = new_data["Squad_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_1'].unique() else 0)
    try:
        new_data["finalist_"+competition[0]] = new_data["Squad_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_2'].unique() else 0)
    except KeyError:
        continue

#4) idem pour cup nationales
for competition in tqdm(competitions_winners.items()):
    new_data["winner_"+competition[0]] = new_data["Nation_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_1'].unique() else 0)
    try:
        new_data["finalist_"+competition[0]] = new_data["Nation_JK"].apply(lambda x : 1 if x in competition[1][competition[0]+'_JK_2'].unique() else 0)
    except KeyError:
        continue

100%|██████████| 11/11 [00:00<00:00, 23.23it/s]


In [155]:
competitions_winners["Liga"]["Liga_JK_1"] = competitions_winners["Liga"]["Liga_JK_1"].apply(lambda x : x.strip())
competitions_winners["Bundesliga"]["Bundesliga_JK_1"] = competitions_winners["Bundesliga"]["Bundesliga_JK_1"].apply(lambda x : x.strip())
competitions_winners["C3"]["C3_JK_1"] = competitions_winners["C3"]["C3_JK_1"].apply(lambda x : x.strip())
competitions_winners["C3"]["C3_JK_2"] = competitions_winners["C3"]["C3_JK_2"].apply(lambda x : x.strip())
competitions_winners["Club WC"]["Club WC_JK_1"] = competitions_winners["Club WC"]["Club WC_JK_1"].apply(lambda x : x.strip())
competitions_winners["Club WC"]["Club WC_JK_2"] = competitions_winners["Club WC"]["Club WC_JK_1"].apply(lambda x : x.strip())
competitions_winners["UCL"]["UCL_JK_1"] = competitions_winners["UCL"]["UCL_JK_1"].apply(lambda x : x.strip())
competitions_winners["UCL"]["UCL_JK_2"] = competitions_winners["UCL"]["UCL_JK_2"].apply(lambda x : x.strip())
competitions_winners["Ligue 1"]["Ligue 1_JK_1"] = competitions_winners["Ligue 1"]["Ligue 1_JK_1"].apply(lambda x : x.strip())
competitions_winners["PL"]["PL_JK_1"] = competitions_winners["PL"]["PL_JK_1"].apply(lambda x : x.strip())
competitions_winners["Serie A"]["Serie A_JK_1"] = competitions_winners["Serie A"]["Serie A_JK_1"].apply(lambda x : x.strip())

In [156]:
new_data["winner_UCL"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['UCL']['UCL_JK_1'].unique() else 0)
new_data["finalist_UCL"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['UCL']['UCL_JK_2'].unique() else 0)
new_data["winner_Bundesliga"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Bundesliga']['Bundesliga_JK_1'].unique() else 0)
new_data["winner_C3"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['C3']['C3_JK_1'].unique() else 0)
new_data["finalist_C3"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['C3']['C3_JK_2'].unique() else 0)
new_data["winner_Club WC"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Club WC']['Club WC_JK_1'].unique() else 0)
new_data["finalist_Club WC"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Club WC']['Club WC_JK_2'].unique() else 0)
new_data["winner_Liga"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Liga']['Liga_JK_1'].unique() else 0)
new_data["winner_Ligue 1"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Ligue 1']['Ligue 1_JK_1'].unique() else 0)
new_data["winner_PL"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['PL']['PL_JK_1'].unique() else 0)
new_data["winner_Serie A"] = new_data["Squad_JK"].apply(lambda x : 1 if x in competitions_winners['Serie A']['Serie A_JK_1'].unique() else 0)

In [157]:
new_data[new_data["Player"] == "Cristiano Ronaldo"][["Player","Squad","Season_End_Year","winner_UCL","winner_Liga","winner_Club WC","finalist_Euro","finalist_UCL"]]

,Player,Squad,Season_End_Year,winner_UCL,winner_Liga,winner_Club WC,finalist_Euro,finalist_UCL
258,Cristiano Ronaldo,Manchester Utd,2004,0,0,0,1,0
298,Cristiano Ronaldo,Manchester Utd,2005,0,0,0,0,0
316,Cristiano Ronaldo,Manchester Utd,2006,0,0,0,0,0
330,Cristiano Ronaldo,Manchester Utd,2007,0,0,0,0,0
364,Cristiano Ronaldo,Manchester Utd,2008,1,0,1,0,0
390,Cristiano Ronaldo,Manchester Utd,2009,0,0,0,0,1
424,Cristiano Ronaldo,Real Madrid,2010,0,0,0,0,0
443,Cristiano Ronaldo,Real Madrid,2011,0,0,0,0,0
466,Cristiano Ronaldo,Real Madrid,2012,0,1,0,0,0
508,Cristiano Ronaldo,Real Madrid,2013,0,0,0,0,0


In [159]:
new_data.to_csv("../data/merged_data.csv",index=False)